In [73]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import transforms
from PIL import Image
from face_recognition.models.FaceNet import get_model, FaceNet
from torchvision.models import resnet50, resnet18


In [11]:
model = get_model()

In [12]:
model_resnet50 = resnet50()

In [66]:
img_path = "./face_recognition/data/images/lfw_aligned/Aaron_Eckhart/Aaron_Eckhart_0001.jpg"
img = Image.open(img_path)
transpose = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
in_tensor = transpose(img)
in_tensor.shape

torch.Size([3, 224, 224])

In [67]:
test = nn.Sequential(
    model_resnet50.conv1,
    model_resnet50.bn1,
    model_resnet50.relu,
    model_resnet50.maxpool,
    model_resnet50.layer1,
    model_resnet50.layer2,
    model_resnet50.layer3,
    model_resnet50.layer4
)

test = test.to("cuda")
sample = in_tensor.unsqueeze(0).to("cuda")

In [68]:
result = test(sample)

print(result.shape)

torch.Size([1, 2048, 7, 7])


In [69]:
model = model.to("cuda")
result_3 = model.cnn(sample)
result_3.shape

torch.Size([1, 2048, 7, 7])

In [74]:
flatten = nn.Flatten().to("cuda")

In [77]:
tmp = model.model.fc(result_3)
l2 = model.l2_norm(tmp)
l2.shape

torch.Size([1, 128])

In [78]:
f_l2 = F.normalize(tmp, p=2, dim=1)
f_l2.shape

torch.Size([1, 128])

In [87]:
class FaceNet2(nn.Module):
    def __init__(self, pretrained=False, embedding_size=128):
        super(FaceNet2, self).__init__()
        resnet = resnet50(pretrained)
        self.backbone = nn.Sequential(
            resnet.conv1,
            resnet.bn1,
            resnet.relu,
            resnet.maxpool,
            resnet.layer1,
            resnet.layer2,
            resnet.layer3,
            resnet.layer4
        )
        in_features = 2048 * 7 * 7 # input image of backbone is of shape (3, 224, 244)
        self.embedder = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features, embedding_size)
        )

    def forward(self, x):
        x = self.backbone(x)
        x = self.embedder(x)
        x = F.normalize(x, p=2, dim=1)
        x = x * 10  # alpha = 10

        return x

In [89]:
tmp_model = FaceNet2()

In [92]:
tmp_model.backbone = model.cnn
tmp_model.embedder = model.model.fc

In [95]:
out_tmp = tmp_model(sample)
out = model(sample)

print(out_tmp.shape)
print(out.shape)

torch.Size([1, 128])
torch.Size([1, 128])


In [96]:
out == out_tmp

tensor([[True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True]], device='cuda:0')

In [129]:
out_path = "./face_recognition/results/models/facesecure_model.pth"
torch.save(tmp_model.state_dict(), out_path)

In [132]:
model = FaceNet2(pretrained=True)

model.load_state_dict(torch.load(out_path))
model.to("cuda")


FaceNet2(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(64

In [133]:
a = model(sample)
b = tmp_model(sample)

In [134]:
a == b

tensor([[True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True]], device='cuda:0')